<a href="https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject_person_reid_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject.ipynb](https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject.ipynb)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/dataset.zip" -d dataset

Output streaming troncato alle ultime 5000 righe.
  inflating: dataset/train/0350_c1_083624030.jpg  
  inflating: dataset/train/1411_c1_004276123.jpg  
  inflating: dataset/train/1411_c1_063612070.jpg  
  inflating: dataset/train/1411_c3_073654330.jpg  
  inflating: dataset/train/1411_c5_065380551.jpg  
  inflating: dataset/train/0959_c1_091954569.jpg  
  inflating: dataset/train/0959_c3_014933988.jpg  
  inflating: dataset/train/0959_c5_006154919.jpg  
  inflating: dataset/train/0959_c6_045481865.jpg  
  inflating: dataset/train/0852_c6_004791953.jpg  
  inflating: dataset/train/1177_c3_011636446.jpg  
  inflating: dataset/train/0474_c1_043109826.jpg  
  inflating: dataset/train/0474_c5_069409220.jpg  
  inflating: dataset/train/0474_c5_013563167.jpg  
  inflating: dataset/train/0474_c5_029799244.jpg  
  inflating: dataset/train/1487_c1_003967228.jpg  
  inflating: dataset/train/1487_c5_046010440.jpg  
  inflating: dataset/train/1487_c5_044022610.jpg  
  inflating: dataset/train/1487_

# Deep Learning Project - People ReID

In [4]:
# import necessary libraries
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import os
from os import listdir
from os.path import isfile, join
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import random
random.seed(10)
# print cuda info
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Cuda device count: {torch.cuda.device_count()}")

Cuda available: True
Cuda device count: 1


# Network
## Siamese Network

In [5]:
 '''
Input arguments
  num_classes: number of classes in the dataset.
               This is equal to the number of output neurons.
'''

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

class Siamese(nn.Module):

    def __init__(self, resnet):
        super(Siamese, self).__init__()
        self.resnet = resnet
        self.resnet.fc = Identity()
        self.linear = torch.nn.Sequential(
          torch.nn.Linear(in_features=2048, out_features=1024),
          torch.nn.Linear(in_features=1024, out_features=512),
          torch.nn.Sigmoid()
        )

    def forward_one(self, x):
        x = self.resnet(x)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        return out1, out2

def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  #alexnet = torchvision.models.alexnet(pretrained=True)
  wide_resnet = torchvision.models.resnet50(pretrained=True)
  num_features = wide_resnet.fc.in_features
  wide_resnet.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=num_features, out_features=1024),
    torch.nn.Linear(in_features=1024, out_features=512),
    torch.nn.Linear(in_features=512, out_features=num_classes),
    torch.nn.Sigmoid()
  )
  #print(resnext)
  # get the number of neurons in the penultimate layer
  #in_features = alexnet.classifier[6].in_features
  
  # re-initalize the output layer
  #alexnet.classifier[6] = torch.nn.Sequential(
  #  torch.nn.Linear(in_features=in_features, out_features=num_classes),
  #  torch.nn.Sigmoid()
  #)
  return wide_resnet

In [6]:
class PeopleValidationDataset(Dataset):
    """People with annotations dataset."""

    def __init__(self, X1, X2, Y, root_dir, transform):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        self.X1 = X1
        self.X2 = X2
        self.Y = Y
        self.root_dir = root_dir

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name1 = self.X1[idx]
        img_name2 = self.X2[idx]

        image1 = Image.open("%s/%s" % (self.root_dir, img_name1))
        image2 = Image.open("%s/%s" % (self.root_dir, img_name2))
        if self.transform != None:
          image1 = self.transform(image1)
          image2 = self.transform(image2)
        else:
          image1 = T.ToTensor()(image1)
          image2 = T.ToTensor()(image2)
        image1 = F.interpolate(image1, size=128)  
        image2 = F.interpolate(image2, size=128)  

        sample = (image1, image2, self.Y[idx])
        return sample

class PeopleTestDataset(Dataset):
    """People with annotations dataset."""

    def __init__(self, X1, X2, query_dir, test_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X1 = X1
        self.X2 = X2
        self.query_dir = query_dir
        self.test_dir = test_dir

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name1 = self.X1[idx]
        img_name2 = self.X2[idx]

        image1 = Image.open("%s/%s" % (self.query_dir, img_name1))
        image2 = Image.open("%s/%s" % (self.test_dir, img_name2))
        image1 = T.ToTensor()(image1)
        image2 = T.ToTensor()(image2)
        image1 = F.interpolate(image1, size=128)  
        image2 = F.interpolate(image2, size=128)  

        sample = (image1, img_name1, image2, img_name2)
        return sample

In [47]:
def split_training_data(root_dir):
  full_dataset_X1 = []
  full_dataset_X2 = []
  full_dataset_Y = []
  img_files = [f for f in listdir(root_dir)]
  img_files.sort()
  #random.shuffle(img_files)
  if(len(img_files) % 2 != 0):
    img_files = img_files[:-1]

  for idx, img in enumerate(img_files):
    if(idx % 2 == 0):
      full_dataset_X1.append(img)
    if(idx % 2 != 0):
      full_dataset_X2.append(img)
      if (int(full_dataset_X1[-1].split("_")[0]) == int(full_dataset_X2[-1].split("_")[0])):
        full_dataset_Y.append(torch.tensor(1))
      else:
        full_dataset_Y.append(torch.tensor(0))

  print(len(full_dataset_X1), len(full_dataset_X2), len(full_dataset_Y))
  val_X1 = []
  val_X2 = []
  val_Y = []
  train_X1 = []
  train_X2 = []
  train_Y = []
  for idx, _ in enumerate(full_dataset_Y):
    if (idx <= len(full_dataset_Y)*0.2):
      val_X1.append(full_dataset_X1[idx])
      val_X2.append(full_dataset_X2[idx])
      val_Y.append(full_dataset_Y[idx])
    else:
      train_X1.append(full_dataset_X1[idx])
      train_X2.append(full_dataset_X2[idx])
      train_Y.append(full_dataset_Y[idx])

  return train_X1, train_X2, train_Y, val_X1, val_X2, val_Y

def build_val_map(val_X1, val_X2):
  val_map = {}
  for i in range(len(val_X1)):
    if int(val_X1[i].split("_")[0]) not in val_map:
      val_map[int(val_X1[i].split("_")[0])] = []
    if int(val_X2[i].split("_")[0]) not in val_map:
      val_map[int(val_X2[i].split("_")[0])] = []
    val_map[int(val_X1[i].split("_")[0])].append(val_X1[i])
    val_map[int(val_X2[i].split("_")[0])].append(val_X2[i])
  for key in val_map:
    val_map[key] = list(set(val_map[key]))
  return val_map

def get_data(batch_size, img_root):
  
  # Prepare data transformations and then combine them sequentially
  # transform = list()
  # transform.append(T.ToTensor())                            # converts Numpy to Pytorch Tensor
  # transform.append(T.Normalize(mean=[0.5], std=[0.5]))      # Normalizes the Tensors between [-1, 1]
  # transform = T.Compose(transform)                          # Composes the above transformations into one.

  # Get splitted data
  train_X1, train_X2, train_Y, val_X1, val_X2, val_Y = split_training_data(root_dir="%s/train" % (img_root))
  
  val_map = build_val_map(val_X1, val_X2)
  val_X1 = []
  val_X2 = []
  val_Y = []
  print(val_map)
  for key in val_map:
    #print(val_map[key])
    for i, value in enumerate(val_map[key]):
      val_Y.append(torch.tensor(1))
      if i == 0:
        val_X1.append([value for _ in range(len(val_map[key])-1)])
      else:
        val_X2.append(value)
  val_X1 = [item for sublist in val_X1 for item in sublist]
  
#validation_data = PeopleValidationDataset(X1=val_X1,
#                                   X2=val_X2,
#                                   Y=val_Y,
#                                   root_dir="%s/train" % (img_root),
#                                   transform=None)
#
#val_loader = torch.utils.data.DataLoader(validation_data, batch_size, shuffle=False, num_workers=0) #before num_workers=4
  
  return val_X1, val_X2

In [45]:
main()

6494 6494 6494
{2: ['0002_c5_040638433.jpg', '0002_c1_016620005.jpg', '0002_c5_048766132.jpg', '0002_c3_027735928.jpg', '0002_c3_093017288.jpg', '0002_c3_010187163.jpg', '0002_c5_089121687.jpg'], 5: ['0005_c2_007791161.jpg', '0005_c2_089100981.jpg', '0005_c5_015923412.jpg', '0005_c4_033112156.jpg', '0005_c4_072835331.jpg', '0005_c5_086279720.jpg', '0005_c6_077255249.jpg', '0005_c1_094238209.jpg', '0005_c2_001852043.jpg', '0005_c5_095652794.jpg', '0005_c6_021313333.jpg', '0005_c5_010064525.jpg', '0005_c5_032936084.jpg', '0005_c3_034151343.jpg', '0005_c3_019884819.jpg', '0005_c1_009701064.jpg', '0005_c3_091344059.jpg', '0005_c6_087351368.jpg', '0005_c6_068305386.jpg', '0005_c5_027746084.jpg', '0005_c5_046533019.jpg', '0005_c1_021227015.jpg', '0005_c1_022428042.jpg', '0005_c2_043723953.jpg', '0005_c3_012765105.jpg', '0005_c4_005828252.jpg'], 6: ['0006_c1_073897328.jpg', '0006_c3_004828240.jpg', '0006_c4_010064843.jpg', '0006_c6_013531057.jpg', '0006_c4_030484566.jpg', '0006_c6_034055253.j

NameError: ignored

In [8]:
def getDataToEvaluate(test_dir, query_dir):
    test_files = [f for f in listdir(test_dir)]
    query_files = [f for f in listdir(query_dir)]

    X1 = []
    X2 = []
    for query in query_files:
        for test in test_files:
            X1.append(query)
            X2.append(test)
    return X1, X2
    
def imshow(img,text=None,should_save=False):
    npimg = img.cpu().numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()   

In [46]:
def test(net, val_X1, val_X2, root_dir threshold , device='cuda:0'):
  predictions = {}
  ground_truth = {}
  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for i in range(len(val_X1)):
        img_name1 = val_X1[i]
        img_name2 = val_X2[i]

        #build ground_truth dict
        if img_name1 not in ground_truth:
          ground_truth[img_name1] = []
        ground_truth[img_name1].append(img_name2)

        image1 = Image.open("%s/%s" % (root_dir, img_name1))
        image2 = Image.open("%s/%s" % (root_dir, img_name2))
        image1 = T.ToTensor()(image1)
        image2 = T.ToTensor()(image2)
        image1 = F.interpolate(image1, size=128)  
        image2 = F.interpolate(image2, size=128)  

        # Load data into GPU
        img1 = img1.to(device)
        img2 = img2.to(device)

        # Forward pass
        output1, output2 = net.forward(img1, img2)
        #print(outputs)
        # Apply the loss
        output1 = torch.squeeze(output1,1)
        output2 = torch.squeeze(output2,1)
        euclidean_distance = F.pairwise_distance(output1, output2)

        if img_name1 not in predictions:
          predictions[img_name1] = []
        if euclidean_distance.item() < threshold:
          predictions[img_name1].append(img_name2)
        

  return predictions, ground_truth

In [10]:
def evaluate_map(predictions, ground_truth):
      '''
      Computes the mAP (https://jonathan-hui.medium.com/map-mean-average-precision-for-object-detection-45c121a31173) of the predictions with respect to the given ground truth
      In person reidentification mAP refers to the mean of the AP over all queries.
      The AP for a query is the area under the precision-recall curve obtained from the list of predictions considering the
      ground truth elements as positives and the other ones as negatives

      :param predictions: dictionary from query filename to list of test image filenames associated with the query ordered
                          from the most to the least confident prediction.
                          Represents the predictions to be evaluated.
      :param ground_truth: dictionary from query filename to set of test image filenames associated with the query
                            Represents the ground truth on which to evaluate predictions.

      :return:
      '''

      m_ap = 0.0
      for current_ground_truth_query, current_ground_truth_query_set in ground_truth.items():

          # No predictions were performed for the current query, AP = 0
          if not current_ground_truth_query in predictions:
              continue

          current_ap = 0.0  # The area under the curve for the current sample
          current_predictions_list = predictions[current_ground_truth_query]

          # Recall increments of this quantity each time a new correct prediction is encountered in the prediction list
          delta_recall = 1.0 / len(current_ground_truth_query_set)

          # Goes through the list of predictions
          encountered_positives = 0
          for idx, current_prediction in enumerate(current_predictions_list):
              # Each time a positive is encountered, compute the current precition and the area under the curve
              # since the last positive
              if current_prediction in current_ground_truth_query_set:
                  encountered_positives += 1
                  current_precision = encountered_positives / (idx + 1)
                  current_ap += current_precision * delta_recall

          m_ap += current_ap

      # Compute mean over all queries
      m_ap /= len(ground_truth)

      return m_ap

In [43]:
def log_values(writer, step, loss, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)

def main(batch_size=128, 
         device='cuda:0', 
         learning_rate=0.001, 
         epochs=50, 
         img_root='./dataset',
         threshold=0.01):
  from torch.utils.tensorboard import SummaryWriter
  writer = SummaryWriter(log_dir="runs/exp4")

  val_loader = get_data(batch_size=batch_size, img_root=img_root)
  # Instantiates the model
  net = initialize_alexnet(num_classes=56)#.to(device)
  net = Siamese(net)
  net.load_state_dict(torch.load("/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/models/siamese_net_reid_resnet50_5epoch.pth"))
  net.to(device)
  
  predictions, ground_truth = test(net, val_loader, threshold)
  print(predictions)
  print(ground_truth)
  #m_ap = evaluate_map(predictions, ground_truth)
  #print(m_ap)
#  query_with_test = {}
#  forward_test = {}
#  forward_query = {}
#  for x0, img_name0,x1, img_name1 in test_dataloader:
#      #x0 = x0.to(device)
#      #x1 = x1.to(device)
#
#      # Preparo lista da printare su txt
#      if img_name0 not in query_with_test:
#          query_with_test[img_name0] = []
#
#      # Forward query element
#      if img_name0 not in forward_query:
#          forward_query[img_name0] = net.forward_one(x0)
#
#      # Forward test element
#      if img_name1 not in forward_test:
#          forward_test[img_name1] = net.forward_one(x1)
#
#      euclidean_distance = F.pairwise_distance(forward_query[img_name0], forward_test[img_name1])
#      print(euclidean_distance.item() < threshold)
#      if euclidean_distance.item() < threshold:
#          query_with_test[img_name0].append(img_name1)
#          if (len(query_with_test[img_name0]) % 100 == 0):
#                print(len(query_with_test[img_name0]))
#          concatenated = torch.cat((x0,x1),0)
#          #print(i, euclidean_distance)
#          imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))
#      # Free GPU memory
#      torch.cuda.empty_cache()
#      import gc
#      gc.collect()
#  print(query_with_test)